import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc

# 한글 폰트 설정
rc("font", family="Malgun Gothic")

# 기본 URL
base_url = "https://finance.naver.com/marketindex/oilDailyQuote.naver"

# 크롤링 결과 저장 리스트
results = []

# 페이지 범위 지정 (예: 1페이지부터 5페이지까지)
for page in range(1, 6):
    # URL에 page 파라미터 추가
    params = {
        "marketindexCd": "OIL_GSL",
        "page": page
    }

    # 요청 보내기
    response = requests.get(base_url, params=params)
    response.raise_for_status()

    # BeautifulSoup으로 파싱
    soup = BeautifulSoup(response.text, "html.parser")

    # 테이블에서 행 데이터 찾기
    rows = soup.select("table.tbl_exchange tbody tr")

    for row in rows:
        # 각 열 데이터 추출
        cols = row.find_all("td")
        if len(cols) == 0:
            continue

        date = cols[0].text.strip()  # 날짜
        price = cols[1].text.strip()  # 가격

        # 결과 리스트에 추가
        results.append({
            "날짜": date,
            "가격": price.replace(",", "")  # 가격의 쉼표 제거
        })

# DataFrame으로 변환
df = pd.DataFrame(results)

# 날짜를 시간 순으로 정렬
df["날짜"] = pd.to_datetime(df["날짜"], format="%Y.%m.%d")  # 점(.) 구분자를 사용한 포맷
df["가격"] = pd.to_numeric(df["가격"])  # 가격을 숫자로 변환
df = df.sort_values("날짜")

# 그래프 그리기
plt.figure(figsize=(12, 6))
plt.plot(df["날짜"], df["가격"], marker="o", linestyle="-", label="휘발유 가격")
plt.title("일별 휘발유 가격", fontsize=16)
plt.xlabel("날짜", fontsize=12)
plt.ylabel("가격 (원)", fontsize=12)
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()

# 그래프 보여주기
plt.show()
